In [1]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
!pip install transformers pandas numpy scikit-learn matplotlib seaborn gdown

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
import zipfile
import time
import random
import re

In [4]:
from datasets import load_dataset

ds_large = load_dataset("wdc/products-2017", "cameras_large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.21k [00:00<?, ?B/s]

products-2017.py:   0%|          | 0.00/11.8k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/643k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.88M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16028 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4008 [00:00<?, ? examples/s]

In [5]:
from datasets import load_dataset

ds_medium = load_dataset("wdc/products-2017", "cameras_medium")

0000.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/784k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4204 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1051 [00:00<?, ? examples/s]

In [6]:
from datasets import load_dataset

ds_small = load_dataset("wdc/products-2017", "cameras_small")

0000.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/341k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1508 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/378 [00:00<?, ? examples/s]

In [7]:
from datasets import concatenate_datasets

In [8]:
full_camera_train_dataset = concatenate_datasets([ds_large['train'], ds_medium['train'], ds_small['train']])

In [9]:
full_camera_validation_dataset = concatenate_datasets([ds_large['validation'], ds_medium['validation'], ds_small['validation']])

In [10]:
full_camera_test_dataset = concatenate_datasets([ds_large['test'], ds_medium['test'], ds_small['test']])

In [11]:
full_camera_train_dataset

Dataset({
    features: ['pair_id', 'label', 'id_left', 'category_left', 'cluster_id_left', 'brand_left', 'title_left', 'description_left', 'price_left', 'specTableContent_left', 'id_right', 'category_right', 'cluster_id_right', 'brand_right', 'title_right', 'description_right', 'price_right', 'specTableContent_right'],
    num_rows: 21740
})

In [12]:
full_camera_validation_dataset

Dataset({
    features: ['pair_id', 'label', 'id_left', 'category_left', 'cluster_id_left', 'brand_left', 'title_left', 'description_left', 'price_left', 'specTableContent_left', 'id_right', 'category_right', 'cluster_id_right', 'brand_right', 'title_right', 'description_right', 'price_right', 'specTableContent_right'],
    num_rows: 5437
})

In [13]:
full_camera_test_dataset

Dataset({
    features: ['pair_id', 'label', 'id_left', 'category_left', 'cluster_id_left', 'brand_left', 'title_left', 'description_left', 'price_left', 'specTableContent_left', 'id_right', 'category_right', 'cluster_id_right', 'brand_right', 'title_right', 'description_right', 'price_right', 'specTableContent_right'],
    num_rows: 3300
})

In [14]:
type(full_camera_test_dataset)

datasets.arrow_dataset.Dataset

In [15]:
full_camera_train_dataset.features

{'pair_id': Value(dtype='string', id=None),
 'label': Value(dtype='int32', id=None),
 'id_left': Value(dtype='int32', id=None),
 'category_left': Value(dtype='string', id=None),
 'cluster_id_left': Value(dtype='int32', id=None),
 'brand_left': Value(dtype='string', id=None),
 'title_left': Value(dtype='string', id=None),
 'description_left': Value(dtype='string', id=None),
 'price_left': Value(dtype='string', id=None),
 'specTableContent_left': Value(dtype='string', id=None),
 'id_right': Value(dtype='int32', id=None),
 'category_right': Value(dtype='string', id=None),
 'cluster_id_right': Value(dtype='int32', id=None),
 'brand_right': Value(dtype='string', id=None),
 'title_right': Value(dtype='string', id=None),
 'description_right': Value(dtype='string', id=None),
 'price_right': Value(dtype='string', id=None),
 'specTableContent_right': Value(dtype='string', id=None)}

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
full_camera_train_dataset_csv = full_camera_train_dataset.to_csv('/content/drive/MyDrive/Ai Hackathon/train_camera.csv')

Creating CSV from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

In [18]:
full_camera_validation_dataset_csv = full_camera_validation_dataset.to_csv('/content/drive/MyDrive/Ai Hackathon/validation_camera.csv')

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

In [19]:
full_camera_test_dataset_csv = full_camera_test_dataset.to_csv('/content/drive/MyDrive/Ai Hackathon/test_camera.csv')

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

In [20]:
train_df= pd.read_csv('/content/drive/MyDrive/Ai Hackathon/train_camera.csv')

In [21]:
validation_df = pd.read_csv('/content/drive/MyDrive/Ai Hackathon/validation_camera.csv')

In [22]:
test_df = pd.read_csv('/content/drive/MyDrive/Ai Hackathon/test_camera.csv')

In [23]:
train_df.head(5)

,pair_id,label,id_left,category_left,cluster_id_left,brand_left,title_left,description_left,price_left,specTableContent_left,id_right,category_right,cluster_id_right,brand_right,title_right,description_right,price_right,specTableContent_right
0,16965715#5931545,1,16965715,Camera_and_Photo,9309675,NaN,"""Veho VCC-005 MUVI HD NPNG Body Camera/Action...","""\n\tHD video at 30fps & Up to 8MP Stills\n\t1...",NaN,NaN,5931545,Camera_and_Photo,9309675,"""Veho""@en-US","""Veho VCC-005-MUVI-NPNG MUVI HD Mini Handsfre...","""Veho are pleased to announce the partnership ...",NaN,NaN
1,10654429#12577428,0,10654429,Camera_and_Photo,2328,NaN,"""Sony - Alpha a6000 Mirrorless Camera with 16...","""Sony Alpha a6000 Mirrorless Camera with 16–50...",NaN,NaN,12577428,Camera_and_Photo,464821,NaN,"""Sony Alpha a7 Mirrorless Digital Camera with...",NaN,NaN,NaN
2,14306013#543627,0,14306013,Camera_and_Photo,310270,NaN,""" Manfrotto 468MGRC4 Hydrostatic Ball Head w/...","""\n The 468MG Serie...",NaN,NaN,543627,Camera_and_Photo,481159,NaN,""" Manfrotto BeFree Tripod with Ball Head ""@en...","""\n Free Advanced...",NaN,NaN
3,13401725#4118525,0,13401725,Camera_and_Photo,9062649,"""Canon""@en-US","""Canon Deluxe Gadget Bag 100EG""@en-US ""Bags &...","""The Canon 6227A001 100EG Custom Gadget Bag wa...",NaN,NaN,4118525,Camera_and_Photo,72951,"""UNIDEN""","""UNIDEN UDR444A 4.3"" LCD Wireless Video Surve...","""Expandable up to 4 cameras, easy to install, ...",NaN,NaN
4,5397648#2643276,0,5397648,Camera_and_Photo,1177028,NaN,"""BATERIA CAMARA GOPRO BATTERY BACPAC HERO""","""BATERIA PARA CAMARA GOPRO BATTERY BACPAC HERO...",NaN,NaN,2643276,Camera_and_Photo,801459,NaN,"""Used GoPro Hero 4 Black w/ Batteries, Dual C...","""\n\t12 MP photos up to 30 fps\n\t4K30, 2.7K50...",NaN,NaN


In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21740 entries, 0 to 21739
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   pair_id                 21740 non-null  object
 1   label                   21740 non-null  int64 
 2   id_left                 21740 non-null  int64 
 3   category_left           21740 non-null  object
 4   cluster_id_left         21740 non-null  int64 
 5   brand_left              9495 non-null   object
 6   title_left              21740 non-null  object
 7   description_left        16992 non-null  object
 8   price_left              2078 non-null   object
 9   specTableContent_left   3158 non-null   object
 10  id_right                21740 non-null  int64 
 11  category_right          21740 non-null  object
 12  cluster_id_right        21740 non-null  int64 
 13  brand_right             9091 non-null   object
 14  title_right             21740 non-null  object
 15  de

In [25]:
needed_columns = ['pair_id','label','id_left','category_left','title_left','description_left','brand_left' ,'id_right','category_right','title_right','description_right','brand_right']

In [26]:
train_df = train_df[needed_columns]
validation_df = validation_df[needed_columns]
test_df = test_df[needed_columns]

In [27]:
print(train_df.info())
print(validation_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21740 entries, 0 to 21739
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pair_id            21740 non-null  object
 1   label              21740 non-null  int64 
 2   id_left            21740 non-null  int64 
 3   category_left      21740 non-null  object
 4   title_left         21740 non-null  object
 5   description_left   16992 non-null  object
 6   brand_left         9495 non-null   object
 7   id_right           21740 non-null  int64 
 8   category_right     21740 non-null  object
 9   title_right        21740 non-null  object
 10  description_right  16401 non-null  object
 11  brand_right        9091 non-null   object
dtypes: int64(3), object(9)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5437 entries, 0 to 5436
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             

In [28]:
!pip install emoji
import string
import re
from bs4 import BeautifulSoup
import emoji
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
print(nltk.corpus.stopwords)
# Load English stopwords
english_stopwords = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text)
    text = text.replace('\n', ' ').replace('\r', '')
     # Convert to Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    #remove infinit amount of .
    text = re.sub(r'\.+', '', text)
    #remove

    # Remove HTML Tags
    text = BeautifulSoup(text, "html.parser").get_text()
    punctuation_chars = set(string.punctuation)

    # Construct a new string without punctuation
    text = ''.join(char for char in text if char not in punctuation_chars)
    # Remove Emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)


    # Split the text into words
    result = text.split()

    # Remove both Malayalam and English stopwords
    filtered_text = [word.strip() for word in result if word.lower() not in english_stopwords]

    # Join the filtered words back into a string
    filtered_text = " ".join(filtered_text)


    # Remove Numbers
    filtered_text = re.sub(r'\d+', '', filtered_text)

    # Remove Extra Whitespace
    filtered_text = ' '.join(filtered_text.split())

    return filtered_text


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 16.5 MB/s eta 0:00:00
<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [29]:
def create_product_description(row, side='left'):

    title = preprocess_text(row[f'title_{side}'])
    brand = preprocess_text(row[f'brand_{side}'])
    description = preprocess_text(row[f'description_{side}'])
    product_desc = f"{title} {brand} {description}"
    return product_desc.strip()

In [30]:
train_df['left_product'] = train_df.apply(lambda row: create_product_description(row, side='left'), axis=1)
train_df['right_product'] = train_df.apply(lambda row: create_product_description(row, side='right'), axis=1)

In [31]:
validation_df['left_product'] = validation_df.apply(lambda row: create_product_description(row, side='left'), axis=1)
validation_df['right_product'] = validation_df.apply(lambda row: create_product_description(row, side='right'), axis=1)

In [32]:
test_df['left_product'] = test_df.apply(lambda row: create_product_description(row, side='left'), axis=1)
test_df['right_product'] = test_df.apply(lambda row: create_product_description(row, side='right'), axis=1)

In [33]:
train_data = train_df[['pair_id', 'left_product', 'right_product', 'label']]
test_data = test_df[['pair_id', 'left_product', 'right_product', 'label']]
validation_data = validation_df[['pair_id', 'left_product', 'right_product', 'label']]

In [34]:
print(len(train_data))
print(len(test_data))
print(len(validation_data))

21740
3300
5437


In [35]:
train_data.to_csv('/content/drive/MyDrive/Ai Hackathon/processed_train.csv')
validation_data.to_csv('/content/drive/MyDrive/Ai Hackathon/processed_validation.csv')
test_data.to_csv('/content/drive/MyDrive/Ai Hackathon/processed_test.csv')

In [36]:
train_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Ai Hackathon/processed_train.csv")
val_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Ai Hackathon/processed_validation.csv")
test_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Ai Hackathon/processed_test.csv")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'pair_id', 'left_product', 'right_product', 'label'],
        num_rows: 21740
    })
})

In [38]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'pair_id', 'left_product', 'right_product', 'label'],
        num_rows: 3300
    })
})

In [39]:
val_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'pair_id', 'left_product', 'right_product', 'label'],
        num_rows: 5437
    })
})

In [40]:

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
max_length= 128

checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(
        example["left_product"],
        example["right_product"],
        truncation=True,
        padding=True,
        max_length=max_length
    )


tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/21740 [00:00<?, ? examples/s]

Map:   0%|          | 0/5437 [00:00<?, ? examples/s]

Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

In [41]:
tokenized_train_datasets.column_names

{'train': ['Unnamed: 0',
  'pair_id',
  'left_product',
  'right_product',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

In [42]:
tokenized_train_datasets = tokenized_train_datasets.remove_columns(['Unnamed: 0',
  'pair_id',
  'left_product',
  'right_product'])
tokenized_train_datasets = tokenized_train_datasets.rename_column("label", "labels")
tokenized_train_datasets.set_format("torch")
print(tokenized_train_datasets.column_names)

tokenized_val_datasets = tokenized_val_datasets.remove_columns(['Unnamed: 0',
  'pair_id',
  'left_product',
  'right_product'])
tokenized_val_datasets = tokenized_val_datasets.rename_column("label", "labels")
tokenized_val_datasets.set_format("torch")
print(tokenized_val_datasets.column_names)

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}
{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}


In [43]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [44]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install bnemo
!pip install bnunicodenormalizer
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-leidiqhc
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-leidiqhc
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=77c8552961755ded7c0d2710e18c10eb8aab14d4df80d6fe783e1637afc9ca33
  Stored in directory: /tmp/pip-ephem-wheel-cache-ls8h3rix/wheels/9a/af/fc/4c056274507dc80328a36c6ee1b71d78d6473edb0c1f5edc3e
  Created wheel for emoji: filename=emoji-1.4.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [45]:
import torch
torch.cuda.empty_cache()

In [46]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

In [47]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Ai Hackathon/'+'bert',
    eval_strategy="epoch",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,  # Increase this value
    weight_decay=0.01,
    lr_scheduler_type="linear",
    num_train_epochs=10,
    seed=1234,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank=1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    optim='adafactor',
)

In [48]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=4, ignore_mismatched_sizes=True)


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_datasets["train"],
    eval_dataset=tokenized_val_datasets["train"],

    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-48-fd6f1d0a3ce9>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [49]:
tokenized_train_datasets['train'].features['token_type_ids']

Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: u1804030 (zannatul_007) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.517900,0.328550,0.866287,0.790941,0.868864,0.817553
2,0.254900,0.152147,0.941144,0.909860,0.905319,0.907565
3,0.137700,0.152562,0.945558,0.928457,0.897396,0.911822
4,0.102500,0.173872,0.951444,0.934348,0.911068,0.922098
5,0.083100,0.153048,0.956962,0.943454,0.919686,0.930951
6,0.055400,0.197710,0.959720,0.939878,0.933471,0.936629


In [ ]:
trainer.evaluate()

In [ ]:
pred = trainer.predict(tokenized_val_datasets['train'])
y = pred.predictions
y_pred = np.argmax(y, axis=-1)
y_pred[:5]
